
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](../images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement du changement climat climatique ou l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, produit par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif du pojet à l'IFÉ ENS de Lyon](https://contrib-tremplin.ens-lyon.fr/) et une copie se trouve dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. 

Les ressources  du projet **peuvent être utilisées dans tout autre environnement compatible**, notamment tous les cahiers d'exercices peuvent être exécutés sur toute machine disposant d'un python3  et des bibliothèques jupyter, jupyterlab, numpy, netcdf4. 

Les données _pré-traitées_ utilisées ci-dessous sont **accessibles  en ligne** sur le [serveur de données géolocalisées](http://geoloc-tremplin.ens-lyon.fr/climato-data/) `opendap` du projet tremplin.

![licence : Creative Commons](../images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche des enseignements autour du changement climatique : mitigation et adaptation au changement

# _Phase 1 : Conversion d'un jeu de données extrait de DRIAS pour des traitements locaux et la production de graphiques ou cartes_

Ce cahier d'exercices `ipython` propose une méthode de conversion d'un jeu de données issues des simulations climatiques de Météofrance. Le travail réalisé ici concerne l'explitation de trois séries de modélisations du CNRM  (RCP 2.6 - 4.5 - 8.5).  Les fichiers commandés et téléchargés sur le site  [DRIAS](http://www.drias-climat.fr/), ne peuvent être (encore) directement servis par le serveur du projet (limitation du logiciel OpenDap). Ce cahier permet de convertir ce qui a été reçu de  [DRIAS](http://www.drias-climat.fr/) et de le convertir pour permettre leur stockage en ligne puis l'utilisation directe depuis le cloud à l'aide des autres cahiers de programmes proposés par le projet sur ce site. 

Les programmes ci-dessous **ne peuvent pas être complètement génériques** et une interface graphique permettant de fournir interactivement les paramètres Ad-Hoc n'a pas encore été développée. Les extractions de [DRIAS](http://www.drias-climat.fr/) sont parfaitement cohérentes  et il suffit pour réutiliser ce cahier d'exercices :

    1. De repérer les **noms des répertoires** où se trouvent les fichiers originaux de Drias, et le répertoire où seront sauvegardés les fichiers transformés et de les remplacer.
    2.  De repérer les **noms des fichiers à traiter** et de les remplacer
    3. De repérer le **nom de la variable principale** et de le remplacer si besoin (À cette fin un bloc de programmes permettant d'imprimer  les paramètres de chaque fichier est proposé il suffit de décommenter les lignes  réalisant une impression).

Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Les données concernent un carré 10 x 10 noeuds de la grille ALADIN centré sur la ville de Lyon. Le format d'entrée et de sortie est celui utilisé par les labos de climatologie, d'océanographie : `netCDF`.  Cet outil requiert l'installation des bibliothèques `netCDF4` et `numpy`.

## Préparation de l'environnement et ouverture du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [1]:
mport netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys
import os
import multiprocessing as mp

## Traitement des données par variables

Importation des données depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise, construction d'un jeu de données en convertissant la totalité des variables codées en 64 bits vers un codage sur 32 bits pour que le serveur opendap les gère convenablement.

Le bloc ci dessous contient la fonction qui modifie le type dela variable et récupère les autres paramètres. Ce code est conçu pour exploiter les extractions standard du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp). 

In [4]:
def convert_file(localpathin, localpathout, driasfilename):

    namein = localpathin + driasfilename
    nameout = localpathout + driasfilename
    # print (namein, '\n',  nameout, '\n\n')
    # open input file
    orig_drias = nc.Dataset(namein, mode='r', format="NETCDF4")
    # open output file
    # for security reasons remove the file bearing this filename
    # ! take care not to loose important data
    try:
        os.remove(nameout)
    except OSError:
        pass
    converted_set = nc.Dataset(nameout, mode='w', format='NETCDF4')

    for thisAttr in orig_drias.ncattrs():
        converted_set.setncattr(thisAttr, orig_drias.getncattr(thisAttr))
    # create dimensions
    for dimens in orig_drias.dimensions.keys():
        this_dim = orig_drias.dimensions[dimens]
        converted_set.createDimension(this_dim.name, this_dim.size)
    # create variables
    for var in orig_drias.variables.keys():
        this_name = orig_drias.variables[var].name
        this_type = orig_drias.variables[var].dtype
        if this_type == 'float64':
            this_type = 'float32'
        this_dim = orig_drias.variables[var].dimensions
        try:
            orig_drias.variables[var]._FillValue
        except AttributeError:
            converted_set.createVariable(this_name, this_type, this_dim)
        else:
            converted_set.createVariable(this_name, this_type, this_dim, orig_drias.variables[var]._FillValue)
        for thisAttr in orig_drias.variables[var].ncattrs():
            converted_set.variables[var].setncattr(thisAttr, orig_drias.variables[var].getncattr(thisAttr))
        converted_set.variables[var][:] = orig_drias.variables[var][:]

    converted_set.close()
    print(driasfilename, '\t\t', datetime.now())

    return driasfilename

## Programme principal 

Dans l'attente de mise en place d'interactivité avec `ipywidgets` ou `flexx` le programme principal reçoit le variables permettant d'effectuer le calcul :
    * Le chemin du répertoire contenant les fichiers issus de [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp)
    * Le chemin du répertoire d'accueil des fichiers convertis (ils portent le même nom que les fichiers d'origine ils doivent donc être stockés dans un autre répertoire
    * Un tableau contenant le nom du fichier et le nom de la variable, par défaut on conserve le nom original mais ce tableau permet éventuellement d'utiliser des noms Ad-Hoc.
    
Sous le bloc de code sont imprimés les noms des fichiers traités et leur horodatation.

In [5]:
# ================================= /parallel execution ========================

i = 0

# pathIn = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/'
# pathOut = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/'
path_in = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/Toulouse-1/'
path_out = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/Toulouse-1/'

Names = ['tasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
         'tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
         'tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
         'tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',
         'huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
         'huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc', 
         'huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
         'huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',
         'prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
         'prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
         'prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
         'prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',
         'rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
         'rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
         'rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
         'rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',
         'tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
         'tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
         'tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
         'tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc']

# ============================== linear execution ==========================
for i, this_name in enumerate(Names):
    convert_file(path_in, path_out, this_name)
    i += 1
# ============================== /linear execution ==========================

huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-14 19:28:39.430987
huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-14 19:28:40.060840
huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-14 19:28:40.672312
huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 		 2019-07-14 19:28:41.261681
prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-14 19:28:41.622691
prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-14 19:28:42.211350
prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-14 19:28:42.799668
prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 		 2019-07-14 19:28:43.381917
rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-14 19:28:43.756822
rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-14 19:28:44.352587
rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-14 19:28:44.932431
rst